# Imports and data preprocessing

In [1]:
import pandas as pd
import numpy as np
import math

train_raw = pd.read_csv("./bank-note/train.csv", header=None).values
train_cols = train_raw.shape[1]
train_rows = train_raw.shape[0]
train_x = np.copy(train_raw)
train_x[:,train_cols - 1] = 1
train_y = train_raw[:, train_cols - 1]
train_y[train_y > 0] = 1
train_y[train_y == 0] = -1

test_raw = pd.read_csv("./bank-note/test.csv", header=None).values
test_cols = test_raw.shape[1]
test_rows = test_raw.shape[0]
test_x = np.copy(test_raw)
test_x[:,test_cols - 1] = 1
test_y = test_raw[:, test_cols - 1]
test_y[test_y > 0] = 1
test_y[test_y == 0] = -1

# Perceptron Implementation

In [2]:
################### standard perceptron methods
def get_error(x,y,w):
    n_rows = x.shape[0]
    predictions = np.sign(np.matmul(x, np.reshape(w, (-1,1))))    # predictions = sign(Wt*x)
    predictions = np.reshape(predictions,(1,-1))
    incorrect_predictions = predictions - y
    count_incorrect_predictions = np.count_nonzero(incorrect_predictions)
    Error = count_incorrect_predictions/ n_rows
    return Error

def Standard_Perceptron(X,Y,learningRate, T):
    rows = X.shape[0]
    cols = X.shape[1]
    w = np.zeros(cols)                              # 1. Initialize w = 0 ∈ ℜn
    indices = np.arange(rows)
    
    for epoch in range(T):                          # 2. For epoch = 1 … T:
        np.random.shuffle(indices)                       #1. Shuffle the data
        x = X[indices,:]
        y = Y[indices]
        for i in range(rows):                            #2. For each training example (xi, yi) ∈ D:
            if np.sum(x[i] * w) * y[i] <= 0:                  #If yi wTxi ≤ 0, update w ← w + r yi xi
                w = w + learningRate * y[i] * x[i]
    return w                                        # 3. Return w

################### voted perceptron methods
def get_Voted_error(x, y,W,C):
    n_rows = x.shape[0]
    W = np.transpose(W)
    predictions = np.sign(np.matmul(np.sign(np.matmul(x, W)), C))
    predictions = np.reshape(predictions,(1,-1))
    incorrect_predictions = predictions - y
    count_incorrect_predictions = np.count_nonzero(incorrect_predictions)
    Error = count_incorrect_predictions/ n_rows
    return Error

def Voted_Perceptron(X,Y,learningRate, T):
    rows = X.shape[0]
    cols = X.shape[1]
    w = np.zeros(cols)                              # 1. Initialize w = 0 ∈ ℜn
    m = np.zeros(cols)                              # 1. Initialize m = 0 ∈ ℜn
    indices = np.arange(rows)
    C = np.array([])                                # c0,c1,..., ck
    W = np.array([])                                # w0,w1,..., wk
    c = 0                                           # init c0 = 0
    for epoch in range(T):                          # 2. For epoch = 1 … T:
        np.random.shuffle(indices)                       #1. Shuffle the data
        x = X[indices,:]
        y = Y[indices]
        for i in range(rows):                            #2. For each training example (xi, yi) ∈ D:
            if np.sum(x[i] * w) * y[i] <= 0:                  # If yi wTxi ≤ 0
                W = np.append(W, w)                             # save wm
                C = np.append(C, c)                             # save cm
                w = w + learningRate * y[i] * x[i]              # update wm+1 ← wm + r yi xi
                c = 1                                           # cm = 1
            else:                                             # else
                c += 1                                          # cm = cm+1
    W = np.reshape(W, (C.shape[0],-1))
    C = np.reshape(C, (-1,1))
    return W,C                                        # 3. Return (w1, c1), (w2, c2), …, (wk, Ck)

################### average perceptron methods
def Avg_Perceptron(X,Y,learningRate, T):
    rows = X.shape[0]
    cols = X.shape[1]
    w = np.zeros(cols)                              # 1. Initialize w = 0 ∈ ℜn
    indices = np.arange(rows)
    a = np.zeros(cols)
    for epoch in range(T):                          # 2. For epoch = 1 … T:
        np.random.shuffle(indices)                       #1. Shuffle the data
        x = X[indices,:]
        y = Y[indices]
        for i in range(rows):                            #2. For each training example (xi, yi) ∈ D:
            if np.sum(x[i] * w) * y[i] <= 0:                  #If yi wTxi ≤ 0, update w ← w + r yi xi
                w = w + learningRate * y[i] * x[i]
            a = a + w
    return a                                        # 3. Return w

# Main

## Standard Perceptron

In [3]:
########### Standard Perceptron ##############
w = Standard_Perceptron(train_x, train_y, 0.5, 10)
print(w)
print('Standard Perceptron Test Error: ', get_error(test_x, test_y,w))

[-31.494896  -23.2750689 -19.57994    -2.49841    27.       ]
Standard Perceptron Test Error:  0.028


## Voted Perceptron

In [4]:
########### Voted Perceptron ##############
W,C = Voted_Perceptron(train_x, train_y, 0.5, 10)
np.savetxt("2b.csv", W, delimiter=",")
print("W length:",len(W))
print("W:\n",W)
print("C length:",len(C))
print("C:\n",np.reshape(C, (1,-1))[0])
print('Voted Perceptron Test Error: ', get_Voted_error(test_x, test_y,W,C))

W length: 227
W:
 [[  0.          0.          0.          0.          0.       ]
 [  0.18019    -2.0579     -1.55715     0.185995   -0.5      ]
 [ -1.76741    -0.15005    -1.71367    -1.723705    0.       ]
 ...
 [-25.203405  -19.527715  -21.114715   -5.2988178  26.       ]
 [-25.463155  -17.896065  -22.659465   -4.8063678  25.5      ]
 [-27.311205  -24.735015  -13.869715   -6.1154178  26.       ]]
C length: 227
C:
 [  0.   2.   1.   2.   1.   2.   3.   9.   2.   6.  25.   7.  10.  22.
   1.   4.   1.   3.   1.  10.   6.  18.  13.  28.   3.  38.   6.  28.
  28.  47.  13.  45.  11.   8.   1.   5.   4.  18.   7.   2.  11.  24.
   9.  65. 165.  52.  14.   8.  11.  11.  36.   7.   9.  23.   8.  23.
   3.   8.   8.   1.   7.  18. 112.   3.  47.  13.  49.  32.  23.  51.
   9.  35.   7.  12.  25.  42.  36.   3.   5.   9.  16.  18. 140.  14.
  13.  58. 123.   8.   9.  40.  49.  94.  20.  17.   4.  85. 134.  10.
   3.   8. 103.  50.   5. 136.  63.  32.  88.  33.  33.  14.   7.   3.
  49.  15.  

## Average Perceptron

In [5]:
########### Average Perceptron ##############
w = Avg_Perceptron(train_x, train_y, 0.5, 10)
print(w)
print('Average Perceptron Test Error: ', get_error(test_x, test_y,w))

[-180604.297364 -120519.22208  -124178.083065  -30430.585706
  168434.5     ]
Average Perceptron Test Error:  0.012
